This notebook is part of [miubrt](https://github.com/meteo-ubonn/miubrt).

Copyright (c) [miubrt developers](https://github.com/meteo-ubonn/miubrt/blob/main/CONTRIBUTORS.txt). 
Distributed under the MIT License. See [LICENSE.txt](https://github.com/meteo-ubonn/miubrt/blob/main/LICENSE.txt) for more info.

# Process netcdf4 X-band radar data

We will use the netcdf4 file for one elevation angle. The filter uses a threshold value of RHOHV and melting layer height for masking, which the user needs to provide. Depending on the threshold value, the filter will remove ground clutter and non-metorological signals. 

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import dask
import glob
import wradlib as wrl
import xarray as xr

In [ ]:
# for using local development version, uncomment next lines otherwise installed `miubrt` version will be used
miubrt_dir = "../../"
if sys.path[0] != miubrt_dir:
     sys.path.insert(0, "../../")

In [ ]:
import miubrt as mrt

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
from matplotlib import gridspec
import hvplot
import hvplot.xarray
import holoviews as hv

## User Input

In [ ]:
npath     = "/data/hubhome/pshrestha/netcdf4/20150705"
npath     = "/data/hubhome/pshrestha/netcdf4/20160513"
npath     = "/data/hubhome/pshrestha/netcdf4/20170706"
location  = "BoXPol"
elev_id   = 0
rhohv_th  = 0.7
z_melt    = 4300. #4500.
z_low     = 1000.
z_up      = 7000.

## Load, Dask and Georeference netcdf4 data

In [ ]:
fil_paths = sorted(glob.glob(npath+"/" +location + "*"))
fil_path  = fil_paths[elev_id]
swp = xr.open_dataset(fil_path, chunks={"time": 12})
#
swp = swp.pipe(wrl.georef.georeference_dataset)
swp = swp.chunk()
print('Processed netcdf4 file ', fil_path)
#
#Some JuxPol data have time also as a dimension for elevation and z co-ordinate
nsize = len(swp.elevation.dims)
if nsize == 1:
    elv_angle = '%.2f' %swp.elevation.values[0]
    z = swp['z']
elif nsize==2:
    elv_angle = '%.2f' %swp.elevation.values[0,0]
    z2 = swp['z']
    z  = z2[0,:,:]
elv_str   = swp.elevation.standard_name + ' ' + elv_angle + ' ' + swp.elevation.units
print(elv_str)

## Filter polarimetric moments

In [ ]:
i=1
if (i==0):
    zh_u  = swp['DBZH'].where((swp.z > z_melt))
    zh    = swp['DBZH'].where((swp.RHOHV >= rhohv_th) & (swp.z <= z_melt),zh_u)
    zdr_u = swp['ZDR'].where((swp.z > z_melt))
    zdr   = swp['ZDR'].where((swp.RHOHV >= rhohv_th) & (swp.z <= z_melt),zdr_u)
    kdp_u = swp['KDP'].where((swp.z > z_melt))
    kdp   = swp['KDP'].where((swp.RHOHV >= rhohv_th) & (swp.z <= z_melt),kdp_u)
    rhohv_u = swp['RHOHV'].where((swp.z > z_melt))
    rhohv   = swp['RHOHV'].where((swp.RHOHV >= rhohv_th) & (swp.z <= z_melt),rhohv_u)
    phidp_u = swp['PHIDP'].where((swp.z > z_melt))
    phidp   = swp['PHIDP'].where((swp.RHOHV >= rhohv_th) & (swp.z <= z_melt),phidp_u)
elif (i==1):
    zh    = swp['DBZH']
    zdr   = swp['ZDR']
    kdp   = swp['KDP']
    rhohv   = swp['RHOHV']
    phidp   = swp['PHIDP']
#zh.data

#offset phidp

## Diagnostic Plots

In [ ]:
colors = ('#00f8f8', '#01b8fb', '#0000fa', '#00ff00', '#00d000', '#009400', 
          '#ffff00', '#f0cc00', '#ff9e00', '#ff0000', '#e40000', '#cc0000', 
          '#ff00ff', '#a06cd5')

dbz_levels=  [-10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
zdr_levels = [-1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0]
kdp_levels = [-1., -0.5, -0.1, 0.0, 0.05, 0.10, 0.20, 0.30, 0.40, 0.60, 0.80, 1., 2., 3.]
rho_levels = [0.2,0.4,0.5, 0.7, 0.8, 0.94, 0.95, 0.96, 0.97, 0.98, 0.985, 0.99, 0.995, 1.0] 
#0.7,0.8, 0.85, 0.9, 0.92
#phi_levels = [0, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]
#phi_levels = [100, 105, 110, 115, 120, 125, 130, 140, 150, 160, 170, 180, 190, 200]
phi_levels = [-100, -90, -80, -70, -60, -50, -40, -30, 25, -20, -15, -10, -5, 0]

## Static Plots

In [ ]:
# Check for radar beam elevation
#fig, ax = plt.subplots()
#ax.plot(swp.range, z[0,:])
#ax.set(xlabel='X (km)', ylabel='Z (km)',
#       title=elv_str)
#ax.grid()

In [ ]:
# Polarimetric moments
# For static plots, use standard time
t    = 43 #14
boxminmax = [-10000,130000,-60000,80000]
#boxminmax = [-30000,60000,-60000,40000]
#JUXboxminmax = [-50000,20000,-50000,20000]
#boxminmax =  [-10000,0,-50000,-40000] #zoom JuxPol elev_id=3, t=14
#JUX boxminmax = [-20000,50000,-35000,35000]
#boxminmax = [-10000,70000,0,80000]
xmin = boxminmax[0]
xmax = boxminmax[1]
ymin = boxminmax[2]
ymax = boxminmax[3]

title_string = swp.time.values[t]
fig = plt.figure(figsize=(20,15),constrained_layout=True)
gs = gridspec.GridSpec(2, 2,fig)
#
ax0 = fig.add_subplot(gs[0,0])
#p0  = zh[t].plot.pcolormesh(
p0 = zh[t].wradlib.plot_ppi(cmap=False,
                               levels=dbz_levels,
                               colors=colors, extend='neither',
                               add_colorbar=True, 
                               cbar_kwargs={'ticks': dbz_levels[0:-1]},
                               ax=ax0
                              )

ax0.set_aspect(1)
ax0.set_xlim([xmin,xmax])
ax0.set_ylim([ymin,ymax])
z.plot.contour(x='x', y='y',levels = [z_low,z_melt,z_up],
                 colors=('k',),linestyles=('--',),linewidths=(2,))
plt.title(title_string)


#
#
ax1 = fig.add_subplot(gs[0,1:])
p1  = zdr[t].wradlib.plot_ppi(cmap=False,
                               levels=zdr_levels,
                               colors=colors, extend='neither',
                               add_colorbar=True, 
                               cbar_kwargs={'ticks': zdr_levels[0:-1]},
                               ax=ax1
                            )

ax1.set_aspect(1)
ax1.set_xlim([xmin,xmax])
ax1.set_ylim([ymin,ymax])
zh[t].plot.contour(x='x', y='y',levels = [20],
                 colors=('grey',),linestyles=('-',),linewidths=(1,))
z.plot.contour(x='x', y='y',levels = [z_low,z_melt,z_up],
                 colors=('k',),linestyles=('--',),linewidths=(2,))
plt.title(title_string)

#
ax2 = fig.add_subplot(gs[1:,0])
p2  = rhohv[t].wradlib.plot_ppi(cmap=False, 
                               levels=rho_levels,
                               colors=colors, extend='neither',
                               add_colorbar=True, 
                               cbar_kwargs={'ticks': rho_levels[0:-1]},
                               ax=ax2,
                            )
ax2.set_aspect(1)
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])
fig = plt.gcf()
ax = plt.gca()
z.plot.contour(x='x', y='y',levels = [z_low,z_melt,z_up],
                 colors=('k',),linestyles=('--',),linewidths=(2,))
plt.title(title_string)

#
ax3 = fig.add_subplot(gs[1:,1:])
p3  = phidp[t].wradlib.plot_ppi(cmap=False,
                               levels=phi_levels,
                               colors=colors, extend='neither',
                               add_colorbar=True, 
                               #cbar_kwargs={'ticks': phi_levels[0:-1]},
                               ax=ax3,
                            )
ax3.set_aspect(1)
ax3.set_xlim([xmin,xmax])
ax3.set_ylim([ymin,ymax])
zh[t].plot.contour(x='x', y='y',levels = [20],
                 colors=('grey',),linestyles=('-',),linewidths=(1,))
z.plot.contour(x='x', y='y',levels = [z_low,z_melt,z_up],
                 colors=('k',),linestyles=('--',),linewidths=(2,))
plt.title(title_string)

figname = location + "_" + elv_angle + "_" + str(title_string) + ".png"
plt.savefig(figname)

## Dynamic Plots

In [ ]:
zh_plot = zh.hvplot.quadmesh(groupby='time',x='x', y='y', 
                   frame_width=400, aspect=1, rasterize=True,
                   clim=(min(dbz_levels),max(dbz_levels)), cmap=colors)
zh_plot = zh_plot.options(color_levels=list(dbz_levels))
zh_plot

In [ ]:
zdr_plot = zdr.hvplot.quadmesh(groupby='time',x='x', y='y', 
                   frame_width=400, aspect=1, rasterize=True,
                   clim=(min(zdr_levels),max(zdr_levels)), cmap=colors)
zdr_plot = zdr_plot.options(color_levels=list(zdr_levels))
zdr_plot

In [ ]:
rhohv_plot = rhohv.hvplot.quadmesh(groupby='time',x='x', y='y', 
                   frame_width=300, aspect=1, rasterize=True,
                   clim=(min(rho_levels),max(rho_levels)), cmap=colors)
rhohv_plot = rhohv_plot.options(color_levels=list(rho_levels))
rhohv_plot

In [ ]:
phidp_plot = phidp.hvplot.quadmesh(groupby='time',x='x', y='y', 
                   frame_width=300, aspect=1, rasterize=True,
                   clim=(min(phi_levels),max(phi_levels)), cmap=colors)
phidp_plot = phidp_plot.options(color_levels=list(phi_levels))
phidp_plot